In [1]:
from pathlib import Path

from dask.distributed import Client

from excited_workflow.train_models import carbon_tracker


client = Client()

In [2]:
ds_cb = Path("/data/volume_2/EXCITED_prepped_data/CT2022.flux1x1-monthly.nc")
ds_regions = Path("/data/volume_2/EXCITED_prepped_data/regions.nc")
output_dir = Path("/home/cdonnelly")

desired_data = [
    "biomass",
    "spei",
    "modis",
    "era5_monthly",
    "era5_land_monthly",
    "copernicus_landcover"
]

x_keys = ["d2m", "mslhf", "msshf", "ssr", "str", "t2m", "spei", "NIRv", "skt",
            "stl1", "swvl1", "lccs_class"]
y_key = "bio_flux_opt"

In [3]:
ds_input = carbon_tracker.merge_datasets(desired_data, ds_cb)
ds_input

In [ ]:
ds_na = carbon_tracker.mask_region(ds_regions, ds_cb, ds_input)
ds_na

<xarray.Dataset>
Dimensions:           (time: 240, latitude: 45, longitude: 85)
Coordinates:
  * longitude         (longitude) float64 -139.5 -138.5 -137.5 ... -56.5 -55.5
  * latitude          (latitude) float64 15.5 16.5 17.5 18.5 ... 57.5 58.5 59.5
  * time              (time) datetime64[ns] 2000-01-01 2000-02-01 ... 2019-12-01
Data variables: (12/26)
    bio_flux_opt      (time, latitude, longitude) float64 nan nan ... nan nan
    transcom_regions  (latitude, longitude) float64 nan nan nan ... nan nan nan
    biomass           (time, latitude, longitude) float64 nan nan ... nan nan
    spei              (time, latitude, longitude) float64 nan nan ... nan nan
    NDVI              (time, latitude, longitude) float64 nan nan ... nan nan
    NIRv              (time, latitude, longitude) float64 nan nan ... nan nan
    ...                ...
    stl4              (time, latitude, longitude) float64 nan nan ... nan nan
    swvl1             (time, latitude, longitude) float64 nan nan ... nan nan
    swvl2             (time, latitude, longitude) float64 nan nan ... nan nan
    swvl3             (time, latitude, longitude) float64 nan nan ... nan nan
    swvl4             (time, latitude, longitude) float64 nan nan ... nan nan
    lccs_class        (time, latitude, longitude) float32 nan nan ... nan nan
Attributes:
    averaging_period_length_hours:  744
    email:                          carbontracker.team@noaa.gov
    url:                            http://carbontracker.noaa.gov
    institution:                    NOAA Earth System Research Laboratory
    Conventions:                    CF-1.5
    history:                        Time-stamp: <Orion-login-1.HPC.MsState.Ed...
    NCO:                            netCDF Operators version 4.9.3 (Homepage ...
    version:                        CT2022 1x1 3-hourly fluxes as of 2023-01-...

In [ ]:
carbon_tracker.validate_model(ds_na, 5, x_keys, y_key, output_dir)

,Description,Value
0,Session id,3203
1,Target,bio_flux_opt
2,Target type,Regression
3,Original data shape,"(21325, 13)"
4,Transformed data shape,"(21325, 13)"
5,Transformed train set shape,"(14927, 13)"
6,Transformed test set shape,"(6398, 13)"
7,Numeric features,12
8,Preprocess,True
9,Imputation type,simple


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
lightgbm,Light Gradient Boosting Machine,0.3,0.2,0.5,0.8,0.2,4.2,0.4


,Description,Value
0,Session id,585
1,Target,bio_flux_opt
2,Target type,Regression
3,Original data shape,"(21325, 13)"
4,Transformed data shape,"(21325, 13)"
5,Transformed train set shape,"(14927, 13)"
6,Transformed test set shape,"(6398, 13)"
7,Numeric features,12
8,Preprocess,True
9,Imputation type,simple


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
lightgbm,Light Gradient Boosting Machine,0.3,0.2,0.5,0.7,0.2,3.5,0.1


,Description,Value
0,Session id,1635
1,Target,bio_flux_opt
2,Target type,Regression
3,Original data shape,"(21325, 13)"
4,Transformed data shape,"(21325, 13)"
5,Transformed train set shape,"(14927, 13)"
6,Transformed test set shape,"(6398, 13)"
7,Numeric features,12
8,Preprocess,True
9,Imputation type,simple


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
lightgbm,Light Gradient Boosting Machine,0.3,0.3,0.5,0.7,0.2,3.6,0.1


,Description,Value
0,Session id,3089
1,Target,bio_flux_opt
2,Target type,Regression
3,Original data shape,"(21325, 13)"
4,Transformed data shape,"(21325, 13)"
5,Transformed train set shape,"(14927, 13)"
6,Transformed test set shape,"(6398, 13)"
7,Numeric features,12
8,Preprocess,True
9,Imputation type,simple


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
lightgbm,Light Gradient Boosting Machine,0.3,0.3,0.5,0.7,0.2,4.4,0.1


,Description,Value
0,Session id,8581
1,Target,bio_flux_opt
2,Target type,Regression
3,Original data shape,"(21325, 13)"
4,Transformed data shape,"(21325, 13)"
5,Transformed train set shape,"(14927, 13)"
6,Transformed test set shape,"(6398, 13)"
7,Numeric features,12
8,Preprocess,True
9,Imputation type,simple


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
lightgbm,Light Gradient Boosting Machine,0.3,0.3,0.5,0.7,0.2,4.3,0.1


In [ ]:
onnx_model = carbon_tracker.save_model(ds_na, x_keys, y_key, output_dir)

,Description,Value
0,Session id,553
1,Target,bio_flux_opt
2,Target type,Regression
3,Original data shape,"(26656, 13)"
4,Transformed data shape,"(26656, 13)"
5,Transformed train set shape,"(18659, 13)"
6,Transformed test set shape,"(7997, 13)"
7,Numeric features,12
8,Preprocess,True
9,Imputation type,simple


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
lightgbm,Light Gradient Boosting Machine,0.3,0.3,0.5,0.7,0.2,4.4,0.1


2023-12-05 14:27:18,091 - distributed.utils_perf - WARNING - full garbage collections took 17% CPU time recently (threshold: 10%)
2023-12-05 14:27:18,627 - distributed.utils_perf - WARNING - full garbage collections took 17% CPU time recently (threshold: 10%)
The maximum opset needed by this model is only 8.
